In [1]:
import pandas as pd
import tensorflow as tf
import scipy as sc
from scipy.spatial import distance
import cv2
import numpy as np
import threading
import time
from multiprocessing import Queue
import os
import json
from base64 import b64encode
from typing import (
    Any,
    List,
    Tuple,
)

import align.detect_face    # type: ignore
import facenet              # type: ignore

import matplotlib.pyplot as pl
%matplotlib inline

In [2]:
base_path = "D:/NNA/"
data_path = base_path + "data/"
facenet_path = base_path + "node-facenet/"
model_path = facenet_path + "models/"

minimum_size_of_face = 60

In [3]:
frames = Queue(8)
faces = Queue(8)

In [4]:
class MtcnnBridge():
    """
    MTCNN Face Alignment
    """
    def __init__(self) -> None:
        self.graph = self.session = None            # type: Any
        self.pnet = self.rnet = self.onet = None    # type: Any

        self.minsize = minimum_size_of_face # minimum size of face
        self.threshold = [0.6, 0.7, 0.7]    # three steps's threshold
        self.factor = 0.709                 # scale factor

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                self.pnet, self.rnet, self.onet = \
                    align.detect_face.create_mtcnn(self.session, None)

    def align(
            self,
            image: np.ndarray,
    ) -> Tuple[List[Any], List[Any]]:
        """ doc """

        bounding_boxes, landmarks = align.detect_face.detect_face(
            image,   # get rid of alpha channel(if any)
            self.minsize,
            self.pnet,
            self.rnet,
            self.onet,
            self.threshold,
            self.factor,
        )

        return bounding_boxes.tolist(), landmarks.tolist()

In [5]:
class FacenetBridge(object):
    """
    Bridge of Facenet
    """
    FACENET_MODEL = None   # type: str

    def __init__(self) -> None:
        self.graph = self.session = None        # type: Any

        self.placeholder_input = None           # type: Any
        self.placeholder_phase_train = None     # type: Any
        self.placeholder_embeddings = None      # type: Any

        self.FACENET_MODEL = FacenetBridge.get_model_path()

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                model_dir = os.path.expanduser(self.FACENET_MODEL)
                meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
                saver = tf.train.import_meta_graph(
                    os.path.join(model_dir, meta_file),
                )
                saver.restore(
                    tf.get_default_session(),
                    os.path.join(model_dir, ckpt_file),
                )
                # facenet.load_model(self.FACENET_MODEL)

        self.placeholder_input = self.graph.get_tensor_by_name('input:0')
        self.placeholder_phase_train = \
            self.graph.get_tensor_by_name('phase_train:0')
        self.placeholder_embeddings = \
            self.graph.get_tensor_by_name('embeddings:0')

    @staticmethod
    def get_model_path() -> str:
        return model_path

    def embedding(
            self,
            image: np.ndarray
    ) -> List[float]:
        """
        Get embedding
        """

        # get rid of Alpha Channel from PNG(if any) and prewhiten
        image = facenet.prewhiten(image)

        feed_dict = {
            self.placeholder_input:         image[np.newaxis, :],
            self.placeholder_phase_train:   False,
        }
        # Use the facenet model to calcualte embeddings
        embeddings = self.session.run(
            self.placeholder_embeddings,
            feed_dict=feed_dict,
        )

        # Return the only row
        return embeddings[0].tolist()    
    
    def embeddings(
            self,
            images: np.ndarray
    ) -> np.ndarray:
        """
        Get embedding
        """

        slices = np.empty((len(images), 160, 160, 3))
        for i in range(len(images)):
            slices[i,:,:,:] = facenet.prewhiten(images[i])
            
        feed_dict = {
            self.placeholder_input:         slices,
            self.placeholder_phase_train:   False,
        }
        # Use the facenet model to calcualte embeddings
        embeddings = self.session.run(
            self.placeholder_embeddings,
            feed_dict=feed_dict,
        )

        print("images " + str(len(images)))
        print("emb " + str(len(embeddings)))
        
        # Return the only row
        return embeddings

In [6]:
class MtcnnMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.initNN()

    def initNN(self):
        self.mtcnn = MtcnnBridge()
        self.mtcnn.init()

    def run(self):
        print("Facenet thread: run")
        global frames
        global faces
        
        while True:
            if(not frames.empty()):
                frame = frames.get()                
                areas = self.mtcnn.align(frame[1])[0]                
                areasCount = len(areas)
                
                if (not areasCount == 0):
                    fullSize = frame[0]

                    height = np.size(fullSize, 0)
                    width = np.size(fullSize, 1)

                    positions = np.empty((areasCount, 4))
                    embeddings = np.empty((areasCount, 160, 160, 3))

                    for i in range(len(areas)):
                        pos = areas[i]
                        x = max(int(pos[0] * 2), 0)
                        y = max(int(pos[1] * 2), 0)
                        w = min(int(pos[2] * 2), width)
                        h = min(int(pos[3] * 2), height)

                        img = fullSize[y:h, x:w, 0:3]
                        img = cv2.resize(img, dsize=(160, 160), interpolation=cv2.INTER_CUBIC)                                                            

                        positions[i,:] = [x,y,w,h]
                        embeddings[i,:,:,:] = img

                    if (not faces.full()):
                        faces.put([positions, embeddings])

                    

In [7]:
class FacenetMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.user = []
        self.distances = []        
        self.loadUsers()
        self.initNN()

    def loadUsers(self):
        dataFile = "./data.json"
        file = open(dataFile, "r")
        self.users = json.loads(file.read())
        
    def initNN(self):
        self.face = FacenetBridge()
        self.face.init()

    def trackSessions(self, emb):
        nearDistances = []
        for user in self.users:
            userEmb = user['embedding']        
            userName = user['name']

            dist = distance.euclidean(userEmb, emb)
            if dist < 1.0:
                nearDistances.append((userName, dist))
                
        if(len(nearDistances) > 0):
            newlist = sorted(nearDistances, key=lambda x: x[1]) 

            print(newlist[0][0] + " - " + str(newlist[0][1]))
            
            self.distances.append(newlist[0][1])
        
        
    def run(self):
        print("Facenet thread: run")
        
        global faces
        
        while True:
            if(not faces.empty()):
                facesOnScreen = faces.get()                
                positions = facesOnScreen[0]
                images = facesOnScreen[1]
                                
                embeddings = self.face.embeddings(images)
                
                for emb in embeddings:                    
                    self.trackSessions(emb)                
                                        

In [8]:
class ImageGrabber(threading.Thread):
    def __init__(self, ID):
        threading.Thread.__init__(self)
        self.ID=ID
        self.cam=cv2.VideoCapture(ID)

    def run(self):
        print("Image grabber thread: run")
        
        global frames
        
        while True:
            ret, frame=self.cam.read()
            if frames.qsize() < 4:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                height = np.size(frame, 0)
                width = np.size(frame, 1)
                frame2 = cv2.resize(frame, dsize=(int(width / 2), int(height / 2)), interpolation=cv2.INTER_CUBIC)
                
                if (not frames.full()):
                    frames.put([frame, frame2])
            #else:
                #print('Skip frame')
                
            

In [ ]:
grabber = ImageGrabber(0)
#grabber = ImageGrabber("rtsp://guest:sup3rGu3st@@10.10.101.156/ch1_1.h264")

mtcnn = MtcnnMain()
mtcnn2 = MtcnnMain()

fn = FacenetMain()
fn2 = FacenetMain()

grabber.start()
mtcnn.start()
mtcnn2.start()
fn.start()
fn2.start()

fn2.join()
fn.join()
mtcnn.join()
mtcnn2.join()
grabber.join()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from D:/NNA/node-facenet/models/model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from D:/NNA/node-facenet/models/model-20170512-110547.ckpt-250000
Image grabber thread: run
Facenet thread: run
Facenet thread: run
Facenet thread: run
Facenet thread: run
images 1
emb 1
Oleksii Makarov - 0.7673924886909081
images 1
emb 1
Oleksii Makarov - 0.7758204870268153
images 1
emb 1
Oleksii Makarov - 0.8294784488771517
images 1
emb 1
Oleksii Makarov - 0.8267916708421927
images 1
emb 1
Oleksii Makarov - 0.7592179995655726
images 1
emb 1
Oleksii Makarov - 0.7928707191158622
images 1
emb 1
Oleksii Makarov - 0.7882673710858307
images 1
emb 1
images 1
emb 1
Oleksii Makarov - 0.8013448222179017
Oleksii Makarov - 0.7731840174340884
images 1
emb 1
Oleksii Makar

images 1
emb 1
images 1
Oleksii Makarov - 0.6807860489296504emb 1

Oleksii Makarov - 0.6943998200620389
images 1
emb 1
Oleksii Makarov - 0.6854242680632985
images 1
emb 1
Oleksii Makarov - 0.6729081902400277
images 1
emb 1
Oleksii Makarov - 0.6521108404571021
images 1
emb 1
Oleksii Makarov - 0.7355131311160622
images 1
emb 1
Oleksii Makarov - 0.7294982496089303
images 1
emb 1
Oleksii Makarov - 0.6252054056981243
images 1
emb 1
Oleksii Makarov - 0.712375959160018
images 1
emb 1
Oleksii Makarov - 0.6917162976069716
images 1
emb 1
Oleksii Makarov - 0.6283917031513838
images 1
emb 1
Oleksii Makarov - 0.6945036550566671
images 1
emb 1
images 1
emb 1
Oleksii Makarov - 0.6266419089280149
Oleksii Makarov - 0.7063368850915345
images 1
emb 1
Oleksii Makarov - 0.6321077710687127
images 1
emb 1
Oleksii Makarov - 0.7461141291450034
images 1
emb 1
Oleksii Makarov - 0.7032847803984917
images 1
emb 1
Oleksii Makarov - 0.6690148601312417
images 1
emb 1
Oleksii Makarov - 0.6784946245777029
images 1
emb 

images 1
emb 1
Oleksii Makarov - 0.7527409943235343
images 1
emb 1
Oleksii Makarov - 0.6643775330471179
images 1
emb 1
Oleksii Makarov - 0.7666421043797288
images 1
emb 1
Oleksii Makarov - 0.6630270579268351
images 1
emb 1
images 1
emb 1
Oleksii Makarov - 0.7501211617882092Oleksii Makarov - 0.7357902900761569

images 1
emb 1
images 1
Oleksii Makarov - 0.6839779140234749emb 1
Oleksii Makarov - 0.660897690066959

images 1
emb 1
images 1
Oleksii Makarov - 0.6689604538474795emb 1

Oleksii Makarov - 0.7500425800379936
images 1
emb 1
Oleksii Makarov - 0.6682483998392397
images 1
emb 1
Oleksii Makarov - 0.6948718736414768
images 1
emb 1
Oleksii Makarov - 0.6949910343407093
images 1
emb 1
Oleksii Makarov - 0.6936284527072092
images 1
emb 1
Oleksii Makarov - 0.6877268633810435
images 1
emb 1
Oleksii Makarov - 0.6836971274653707
images 1
emb 1
Oleksii Makarov - 0.6719777306435075images 1

emb 1
Oleksii Makarov - 0.6805573733892126
images 1
emb 1
Oleksii Makarov - 0.6260825049020627
images 1
emb 

images 1
emb 1
Oleksii Makarov - 0.7902913875388397
images 1
emb 1
Oleksii Makarov - 0.7310472030675143
images 1
emb 1
Oleksii Makarov - 0.8016135672252375
images 1
emb 1
Oleksii Makarov - 0.789036392798089
images 1
emb 1
Oleksii Makarov - 0.7586777490232608
images 1
emb 1
Oleksii Makarov - 0.787385155849143
images 1
emb 1
Oleksii Makarov - 0.7808439524199364
images 1
emb 1
Oleksii Makarov - 0.8172471312764198
images 1
emb 1
Oleksii Makarov - 0.83851491826275
images 1
emb 1
Oleksii Makarov - 0.7378575960266064
images 1
emb 1
Oleksii Makarov - 0.7229448770636224
images 1
emb 1
Oleksii Makarov - 0.7253106513622317
images 1
emb 1
Oleksii Makarov - 0.7748222142442894
images 1
emb 1
Oleksii Makarov - 0.7490499398324545
